# 多输出模型实例

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from glob import glob

In [2]:
# 数据集准备
data_dir = r'D:\dataset\multi-output-classification\dataset'
colors = ['black', 'blue', 'red']
coats = ['jeans', 'shoes', 'dress', 'shirt']
all_dirs = os.listdir(data_dir)

In [3]:
image_filenames = []
color_labels = []
coat_labels = []
for each_dir in all_dirs:
    color, coat = each_dir.split('_')
    color_id, coat_id = colors.index(color), coats.index(coat)
    image_docunts = len(os.listdir(os.path.join(data_dir, each_dir)))
    color_labels.extend([[color_id]] * image_docunts)
    coat_labels.extend([[coat_id]] * image_docunts)
    image_filenames.extend(glob(os.path.join(data_dir, each_dir, '*')))

In [4]:
def preprocessing(x, y):
    image = tf.io.read_file(x)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, size=(height, width))
#     image = tf.keras.applications.inception_v3.preprocess_input(image)
    image = image / 255.0 * 2 - 1
    return image, y

height = 224
width = 224
channels = 3
batch_size = 8


dataset = tf.data.Dataset.from_tensor_slices((image_filenames, (color_labels, coat_labels)))

In [5]:
# 分割训练集和测试集
train_count = int(len(image_filenames) * 0.8)
train_dataset = dataset.take(train_count)
valid_dataset = dataset.skip(train_count)

train_dataset = train_dataset.shuffle(train_count).map(preprocessing).batch(batch_size).repeat()
valid_dataset = valid_dataset.map(preprocessing).batch(batch_size)

In [6]:
# 构建模型
# base = tf.keras.applications.inception_v3.InceptionV3(
#     include_top=False, weights='imagenet', input_shape=(height, width, channels),
#     pooling='avg'
# )
# base = tf.keras.applications.ResNet50(
#     include_top=False, weights='imagenet', input_shape=(height, width, channels),
#     pooling='avg'
# )
base = tf.keras.applications.MobileNetV2(
    include_top=False, weights='imagenet', input_shape=(height, width, channels),
    pooling='avg'
)

base.trainable = True
x = keras.layers.Input(shape=(height, width, channels))
out = base(x)
# out1为颜色分类输出
# out1 = keras.layers.Dense(512, activation='relu')(out)
# out1 = keras.layers.Dense(2048, activation='relu')(out1)
out1 = keras.layers.Dense(len(colors), activation='softmax')(out)
# out2为衣服种类分类输出
# out2 = keras.layers.Dense(1024, activation='relu')(out)
out2 = keras.layers.Dense(len(coats), activation='softmax')(out)

model = keras.models.Model(x, (out1, out2))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
mobilenetv2_1.00_224 (Model)    (None, 1280)         2257984     input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 3)            3843        mobilenetv2_1.00_224[1][0]       
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 4)            5124        mobilenetv2_1.00_224[1][0]       
Total params: 2,266,951
Trainable params: 2,232,839
Non-trainable params: 34,112
_____________

In [7]:
sample = train_dataset.take(1).as_numpy_iterator().next()[0]

In [8]:
# 配置模型
model.compile(
    loss=keras.losses.sparse_categorical_crossentropy,
    optimizer=keras.optimizers.Adam(lr=1e-4),
    metrics=['accuracy']
)

callbacks = [
    keras.callbacks.EarlyStopping(patience=15),
    keras.callbacks.ReduceLROnPlateau(patience=10),
    keras.callbacks.ModelCheckpoint('./multi_classifier.tf', save_best_only=True)
]

In [10]:
model.fit(train_dataset, epochs=100, steps_per_epoch=train_count // batch_size,
          validation_data=valid_dataset, callbacks=callbacks
         )

Epoch 1/100
252/252 [==============================] - 43s 171ms/step - loss: 0.0075 - dense_loss: 0.0040 - dense_1_loss: 0.0035 - dense_accuracy: 0.9995 - dense_1_accuracy: 0.9985 - val_loss: 0.6917 - val_dense_loss: 0.3265 - val_dense_1_loss: 0.3652 - val_dense_accuracy: 0.8812 - val_dense_1_accuracy: 0.9228 - lr: 1.0000e-05
Epoch 2/100
252/252 [==============================] - 42s 167ms/step - loss: 0.0086 - dense_loss: 0.0046 - dense_1_loss: 0.0040 - dense_accuracy: 0.9980 - dense_1_accuracy: 0.9985 - val_loss: 0.7213 - val_dense_loss: 0.4521 - val_dense_1_loss: 0.2691 - val_dense_accuracy: 0.8337 - val_dense_1_accuracy: 0.9406 - lr: 1.0000e-05
Epoch 3/100
252/252 [==============================] - 42s 168ms/step - loss: 0.0029 - dense_loss: 0.0016 - dense_1_loss: 0.0012 - dense_accuracy: 1.0000 - dense_1_accuracy: 1.0000 - val_loss: 0.7846 - val_dense_loss: 0.5264 - val_dense_1_loss: 0.2582 - val_dense_accuracy: 0.8198 - val_dense_1_accuracy: 0.9446 - lr: 1.0000e-05
Epoch 4/100
2